In [ ]:
import torch
import torchvision
import numpy as np
from torchvision import datasets,transforms
from torch.nn import Conv2d, MaxPool2d, Linear, Module, Dropout, CrossEntropyLoss
from torch.nn import functional as F

In [3]:
trainset = torchvision.datasets.MNIST(root='./data/MNIST', train=True, download=True, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),]))
testset = torchvision.datasets.MNIST(root='./data/MNIST', train=False, download=True, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),]))
train = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
test = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [4]:
class Net(Module):
    def __init__(self):
        super(Net,self).__init__()
        self.L1=Conv2d(in_channels=1, out_channels=32, kernel_size=(3,3))
        self.L2=Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3))
        self.L3=MaxPool2d(kernel_size=(2,2))
        self.L4=Dropout(p=0.25)
        self.L5=Linear(in_features=9216, out_features=128)
        self.L6=Dropout()
        self.L7=Linear(in_features=128, out_features=10)
    def forward(self,x):
        x = self.L1(x)
        x = F.relu(x)
        x = self.L2(x)
        x = F.relu(x)
        x = self.L3(x)
        x = self.L4(x)
        x = x.view(-1,9216)
        x = self.L5(x)
        x = F.relu(x)
        x = self.L6(x)
        x = self.L7(x)
        x = F.softmax(x, dim=1)
        return x


In [5]:
net=Net()

In [6]:
import torch.optim as optim
criterion = CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
import time
start_time=time.time()

In [ ]:
for epoch in range(25):
    running_loss = 0.0
    for i, data in enumerate(train, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(epoch + 1, 'loss =', running_loss/len(train))
print('Finished Training')

1 loss = 2.2123342052197406
2 loss = 1.7413594112721587
3 loss = 1.6446112116008424
4 loss = 1.6190547101787414
5 loss = 1.6032774238698264
6 loss = 1.5973011475127896


In [ ]:
gpu_time=time.time()-start_time
gpu_time

3263.4106233119965

In [ ]:
cpu_time=time.time()-start_time
cpu_time

3446.9949724674225

In [ ]:
correct_count, all_count = 0, 0
for images,labels in test:
    with torch.no_grad():
        pred = net(images)
    pred = pred.numpy()
    for i in range(len(labels)):
        pred_label = np.argmax(pred[i])
        true_label = labels.numpy()[i]
        if(true_label == pred_label):
            correct_count += 1
        all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count*100/all_count),'%')

Number Of Images Tested = 10000

Model Accuracy = 96.25 %
